In [6]:
import pandas as pd
import datetime
import requests
from ipywidgets import interactive, widgets, VBox, HBox
pd.options.display.max_rows = None

In [25]:
base_url = 'https://cdn-api.co-vin.in/api'
generate_otp_url = f'{base_url}/v2/auth/public/generateOTP'
confirm_otp_url = f'{base_url}/v2/auth/public/confirmOTP'
states_url = f'{base_url}/v2/admin/location/states'
districts_url = f'{base_url}/v2/admin/location/districts/{{state_id}}'
find_by_district_url = f'{base_url}/v2/appointment/sessions/public/findByDistrict?district_id={{district_id}}&date={{date}}'
calender_by_district_url = f'{base_url}/v2/appointment/sessions/public/calendarByDistrict?district_id={{district_id}}&date={{date}}'
headers = {
#     Host: 'cdn-api.co-vin.in',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
    }
all_cols = ['center_id', 'name', 'address', 'state_name', 'district_name',
            'block_name', 'pincode', 'lat', 'long', 'from', 'to', 'fee_type',
            'session_id', 'date', 'available_capacity', 'min_age_limit', 'vaccine',
            'slots', 'available_capacity_dose1', 'available_capacity_dose2']
display_cols = ['block_name', 'district_name', 'name', 'address', 
                'pincode', 'fee_type', 'date', 'min_age_limit', 'vaccine', 
                'available_capacity_dose1', 'available_capacity_dose2']
date_today = datetime.date.today()
STATE_ID = None
RESULT_DF = None


def get_state_ids():
    res = requests.get(states_url, headers=headers)
    if res.status_code == 200:
        states_df = pd.DataFrame(res.json()['states'], columns=['state_id', 'state_name'])
        return states_df
    print(res.status_code, res.text)
    return pd.DataFrame(columns=['state_id', 'state_name'])

def get_slots_df(state_id, date=date_today.strftime("%d-%m-%Y"), **kwargs):
    '''
    state_id: int
    date: basestring
        format- "%d-%m-%Y"
    kwargs: further filters    
        vaccine: basestring
            'COVAXIN', 'COVISHIELD'
        min_age_limit: int
            18, 45
        fee_type: basestring
            Free, Paid
        
    '''
    res = requests.get(districts_url.format(state_id=state_id), headers=headers)
    if res.status_code != 200:    
        print(res.status_code, res.text)
        return pd.DataFrame(columns=['district_id', 'district_name']), pd.DataFrame(columns=all_cols)
    district_df = pd.DataFrame(res.json()['districts'], columns=['district_id', 'district_name'])
    df_list = []
    for district_id in district_df['district_id']:
        res = requests.get(calender_by_district_url.format(district_id=district_id, date=date), 
                           headers=headers)
        if res.status_code != 200:
            print(res.status_code, res.text)
        centres_df = pd.DataFrame(res.json()['centers'], 
                                  columns=['center_id', 'name', 'address', 'state_name', 'district_name',
                                           'block_name', 'pincode', 'lat', 'long', 'from', 'to', 'fee_type',
                                           'sessions'])
        decoupled_df = centres_df.explode('sessions')
        centre_slots_df = pd.concat([decoupled_df.drop(['sessions'], axis=1), 
                                     decoupled_df['sessions'].apply(pd.Series)], axis=1)
        df_list.append(centre_slots_df)
    combined_df = pd.concat(df_list)
    query_string = ' & '.join([f"{col_name} == '{value}'" if type(value)==str else f"{col_name} == {value}"
                               if col_name in combined_df.columns else '' 
                               for col_name, value in kwargs.items()])
    if query_string:
        combined_df = combined_df.query(query_string)
    return district_df, combined_df

In [26]:
states_df = get_state_ids()
if not states_df.empty:
    state_options = list(states_df.apply(lambda ser: (ser['state_name'], ser['state_id']), axis=1))
else:
    state_options = [('Delhi', 9)]
min_age_limit_options = ['All', 18, 45]
fee_type_options = ['All', 'Free', 'Paid']
vaccine_options = ['All', 'COVISHIELD', 'COVAXIN']
state_widget = widgets.Dropdown(options=state_options,
                                value=9,
                                description='State :',
                                disabled=False)    
min_age_limit_widget = widgets.Dropdown(options=min_age_limit_options,
                                value='All',
                                description='Min Age :',
                                disabled=False)  
fee_type_widget = widgets.Dropdown(options=fee_type_options,
                                value='All',
                                description='Fee Type :',
                                disabled=False)
vaccine_widget = widgets.Dropdown(options=vaccine_options,
                                value='All',
                                description='Vaccine :',
                                disabled=False)
avail_dose_1_widget = widgets.IntSlider(value=0, min=0, max=20, step=1,
                                        description='Dose 1 Slots:', disabled=False,
                                        continuous_update=False, orientation='horizontal',
                                        readout=True, readout_format='d')
avail_dose_2_widget = widgets.IntSlider(value=0, min=0, max=20, step=1,
                                        description='Dose 2 Slots:', disabled=False,
                                        continuous_update=False, orientation='horizontal',
                                        readout=True, readout_format='d')

report_output = widgets.Output()

def filter_function(state_id, avail_dose_1, avail_dose_2, **kwargs):
    global STATE_ID
    global RESULT_DF
    global display_cols
    print(f'State_id: {state_id},', 
          ', '.join([f"{key} : {value}" for key, value in kwargs.items()]),
         f'Dose1 Slots: {avail_dose_1},', 
         f'Dose2 Slots: {avail_dose_2},', )
    result_df = RESULT_DF
    if state_id:
        if STATE_ID is None or STATE_ID != state_id:
            district_df, result_df = get_slots_df(state_id=state_id)#, vaccine='COVAXIN', min_age_limit=18)
            RESULT_DF = result_df
            STATE_ID = state_id
    query_string = ' and '.join([f"{col_name} == '{value}'" if type(value)==str else f"{col_name} == {value}"
                               for col_name, value in kwargs.items() if value != 'All'])
    if query_string:
        filtered_df = result_df.query(query_string)
    else:
        filtered_df = result_df.copy()
    if avail_dose_1:
        filtered_df = filtered_df[filtered_df['available_capacity_dose1']>=avail_dose_1]
    if avail_dose_2:
        filtered_df = filtered_df[filtered_df['available_capacity_dose2']>=avail_dose_2]
    with report_output:
        report_output.clear_output()
        display(filtered_df[display_cols])
   
inter = interactive(filter_function, state_id=state_widget, fee_type=fee_type_widget, 
                    min_age_limit=min_age_limit_widget, vaccine=vaccine_widget, avail_dose_1=avail_dose_1_widget,
                    avail_dose_2 = avail_dose_2_widget) 
VBox([inter, report_output])
